In [1]:
# %%
import sys
sys.path.append('..')
# sys.path.append('/system/user/beck/pwbeck/projects/regularization/ml_utilities')
from pathlib import Path
from typing import Union
from ml_utilities.torch_models.base_model import BaseModel
from ml_utilities.torch_models import get_model_class
from omegaconf import OmegaConf

from erank.utils import load_directions_matrix_from_task_sweep
        


/system/apps/userenv/beck/erank/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
run_path = '/system/user/beck/pwbeck/projects/regularization/erank/outputs/f_mnist-same_init-taskidxXXX_230622_162800/outputs/f_mnist-same_init-taskidx0-seed0_230622_162819'

if isinstance(run_path, str):
    run_path = Path(run_path)
# load config
loaded_config = OmegaConf.load(run_path/'.hydra'/'config.yaml')
config = loaded_config.config
# get model class
model_name = config.model.name
model_class = get_model_class(model_name)
# get best epoch
best_epoch_file = run_path / 'best_epoch.txt'
assert best_epoch_file.exists(), 'No best epoch file found.'

In [3]:
with best_epoch_file.open() as f:
    t = f.read()
    print(t)

12


In [4]:
run_path.stem

'f_mnist-same_init-taskidx0-seed0_230622_162819'

In [6]:
p = '/system/user/beck/pwbeck/projects/regularization/erank/outputs/f_mnist-same_init-taskidxXXX_230622_162800/outputs'
dir_matrix = load_directions_matrix_from_task_sweep(p)
dir_matrix.requires_grad_(True)

Loading /system/user/beck/pwbeck/projects/regularization/erank/outputs/f_mnist-same_init-taskidxXXX_230622_162800/outputs/f_mnist-same_init-taskidx4-seed0_230622_162834: 100%|██████████| 8/8 [00:00<00:00, 17.24it/s]


tensor([[-0.0037, -0.0037, -0.0037,  ..., -0.0012,  0.0081, -0.0076],
        [-0.0050, -0.0050, -0.0050,  ..., -0.0021,  0.0076, -0.0060],
        [-0.0086, -0.0086, -0.0082,  ...,  0.0013,  0.0098, -0.0146],
        ...,
        [-0.0056, -0.0056, -0.0056,  ..., -0.0016,  0.0029, -0.0157],
        [-0.0057, -0.0058, -0.0057,  ...,  0.0014,  0.0032, -0.0097],
        [-0.0039, -0.0039, -0.0038,  ..., -0.0030,  0.0025, -0.0100]],
       device='cuda:0', requires_grad=True)

In [7]:
dir_matrix.shape, dir_matrix.device, dir_matrix.requires_grad

(torch.Size([8, 669706]), device(type='cuda', index=0), True)

In [8]:
d = dict(a=1,b=2,c=3)
for x in d:
    print(x)

a
b
c


In [9]:
import torch
t = torch.tensor(0.0)

In [10]:
t.dtype, t.device

(torch.float32, device(type='cpu'))

In [11]:
def erank(matrix_A: torch.Tensor, center_matrix_A: bool=False) -> torch.Tensor:
        """Calculates the effective rank of a matrix.

        Args:
            matrix_A (torch.Tensor): Matrix of shape m x n. 
            center_matrix_A (bool): Center the matrix 

        Returns:
            torch.Tensor: Effective rank of matrix_A
        """
        assert matrix_A.ndim == 2
        _, s, _ = torch.pca_lowrank(matrix_A, center=center_matrix_A, niter=1, q=min(matrix_A.shape[0], matrix_A.shape[1]))
        s = torch.square(s) / (s.shape[0] - 1)

        # normalizes input s -> scale independent!
        return torch.exp(torch.distributions.Categorical(s).entropy())

In [12]:
er = erank(dir_matrix)

In [13]:
er

tensor(5.0574, device='cuda:0', grad_fn=<ExpBackward>)

In [ ]:
# er.backward()

## normalize directions

In [22]:
norm_dir_matrix = dir_matrix / torch.linalg.norm(dir_matrix, ord=2, dim=1, keepdim=True)

In [24]:
torch.linalg.norm(dir_matrix, ord=2, dim=1, keepdim=True), torch.linalg.norm(norm_dir_matrix, ord=2, dim=1, keepdim=True)

(tensor([[10.9480],
         [11.4298],
         [14.0231],
         [11.9096],
         [ 8.6046],
         [11.5020],
         [10.4693],
         [15.7062]], device='cuda:0', grad_fn=<CopyBackwards>),
 tensor([[1.0000],
         [1.0000],
         [1.0000],
         [1.0000],
         [1.0000],
         [1.0000],
         [1.0000],
         [1.0000]], device='cuda:0', grad_fn=<CopyBackwards>))

In [25]:
erank(norm_dir_matrix)

tensor(5.1965, device='cuda:0', grad_fn=<ExpBackward>)

## try different queues

In [ ]:
from queue import Queue

queue = Queue(2)

queue.put(0)
queue.put(2)

In [ ]:
from collections import deque

q = deque(maxlen=2)
q.append(0)
print(len(q))
q.append(1)
q

1


deque([0, 1])